In [3]:
# -*- coding: utf8 -*-
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import re
import pandas as pd

In [4]:
job_name = 'python'
link_hh = 'https://novosibirsk.hh.ru'
link_sj = 'https://www.superjob.ru'
header = {'Accept': '*/*',
          'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0'
          }

In [5]:
def get_sj_vac(link, header, job_name):
    data = list()
    link_sj = link + '/vacancy/search/?keywords=' + job_name + "&geo%5Bt%5D%5B0%5D=4"
    session = requests.session()
    re_vac = re.compile('i.*')
    re_vac_name = re.compile('icMQ_ _1QIBo .*')
    re_salary = re.compile('.*f-test-text-company-item-salary.*')
    for page in range(1, 100):
        print(link_sj + f"&page={page}")
        response = session.get(link_sj + f"&page={page}", headers=header)
        if response.ok:
            html = bs(response.text, 'lxml')
            vac_list = html.find('div', {'id': "app"})
            vac_list = vac_list.find('div', {'style': "display:block"})

            if not vac_list:
                break

            vac_list = vac_list.findChildren('div', {'class': re_vac}, recursive=False)          

            for vac in vac_list:                
                vac_name = vac.find('a', {'class': re_vac_name})
                if vac_name:
                    vac_data = {}
                    vac_data['site'] = link
                    vac_data['job_name'] = vac_name.text
                    vac_data['href'] = link + vac_name['href']
                    vac_sal = vac.find('span', {'class': re_salary})
                    if vac_sal:
                        sal_ = str(vac_sal.text).replace('\xa0', ' ')
                        sal_min = 0
                        sal_max = 0

                        if sal_ == 'По договорённости':
                            sal_min = 0
                        elif sal_.count('—') > 0:
                            sal_min = sal_.split('—')[0]
                            sal_max = sal_.split('—')[1]
                            sal_max = sal_max.replace('₽', '')
                        elif sal_.count('от') > 0:
                            sal_min = sal_.replace('от', '')
                            sal_min = sal_min.replace('₽', '')

                        vac_data['salary_min'] = sal_min
                        vac_data['salary_max'] = sal_max

                    data.append(vac_data)
        else:
            break;

    return data

In [6]:
def get_hh_vac(link, header, job_name):
    data = list()
    link_hh = link + '/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text=' + job_name
    session = requests.session()
    re_vac = re.compile('vacancy-serp.*')
    for page in range(0, 10):
        print(link_hh + f"&page={page}")
        response = session.get(link_hh + f"&page={page}", headers=header)
        if response.ok:
            html = bs(response.text, 'lxml')
            vac_head = html.find('div', {'class': 'vacancy-serp'})
            vac_l = vac_head.findChildren('div', {'class': re_vac}, recursive=False)
           

            for vac in vac_l:
                vac_data = {}

                vac_name = vac.find('div', {'class': 'vacancy-serp-item__info'})
                vac_name = vac_name.find('a')
                vac_data['job_name'] = vac_name.text
                vac_data['href'] = vac_name['href']

                if vac_name['href']:
                    vac_data['site'] = link

                vac_sal = vac.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
                if vac_sal:
                    sal_ = vac_sal.text.replace(' ', '').replace(' ', '')
                    # print(sal_)
                    sal_min = 0
                    sal_max = 0

                    if sal_.count('-') > 0:
                        sal_min = sal_.split('-')[0]
                        sal_max = sal_.split('-')[1]
                        sal_max = sal_max.replace('руб.', '').replace('USD', '')
                    elif sal_.count('от') > 0:
                        sal_min = sal_.replace('от', '')
                        sal_min = sal_min.replace('руб.', '').replace('USD', '')

                    if sal_.count('USD') == 0:
                        vac_data['salary_min'] = sal_min
                        vac_data['salary_max'] = sal_max
                    else:
                        vac_data['salary_min'] = str(sal_min) + "$"
                        vac_data['salary_max'] = str(sal_max) + "$"

                data.append(vac_data)              

        else:
            break
    return data

In [7]:
result = list()
result=result+(get_sj_vac(link_sj, header,job_name))
result=result+ get_hh_vac(link_hh, header, job_name)
print(len(result))

https://www.superjob.ru/vacancy/search/?keywords=python&geo%5Bt%5D%5B0%5D=4&page=1
https://www.superjob.ru/vacancy/search/?keywords=python&geo%5Bt%5D%5B0%5D=4&page=2
https://www.superjob.ru/vacancy/search/?keywords=python&geo%5Bt%5D%5B0%5D=4&page=3
https://novosibirsk.hh.ru/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text=python&page=0
https://novosibirsk.hh.ru/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text=python&page=1
https://novosibirsk.hh.ru/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text=python&page=2
https://novosibirsk.hh.ru/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text=python&page=3
https://novosibirsk.hh.ru/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text=python&page=4
https://novosibirsk.hh.ru/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text=python&page=5
https:/

In [8]:
data = pd.DataFrame(result)

In [9]:
data

,site,job_name,href,salary_min,salary_max
0,https://www.superjob.ru,Разработчик Python,https://www.superjob.ru/vakansii/razrabotchik-...,170 000,200 000
1,https://www.superjob.ru,Ведущий разработчик Python,https://www.superjob.ru/vakansii/veduschij-raz...,150 000,300 000
2,https://www.superjob.ru,Программист-разработчик Python,https://www.superjob.ru/vakansii/programmist-r...,150 000,0
3,https://www.superjob.ru,Программист-разработчик (Python / Django),https://www.superjob.ru/vakansii/programmist-r...,0,0
4,https://www.superjob.ru,Веб-разработчик / Веб-программист,https://www.superjob.ru/vakansii/veb-razrabotc...,0,0
...,...,...,...,...,...
519,https://novosibirsk.hh.ru,Data Engineer (Middle/Senior),https://novosibirsk.hh.ru/vacancy/31429476?que...,NaN,NaN
520,https://novosibirsk.hh.ru,Senior Python developer в музыкальный стриминг...,https://novosibirsk.hh.ru/vacancy/35462805?que...,NaN,NaN
521,https://novosibirsk.hh.ru,Senior Data scientist (Центр по исследованию д...,https://novosibirsk.hh.ru/vacancy/35907224?que...,NaN,NaN
522,https://novosibirsk.hh.ru,DevOps Engineer,https://novosibirsk.hh.ru/vacancy/35413649?que...,0,0
